# 排序模型

通过召回的操作， 我们已经进行了问题规模的缩减， 对于每个用户， 选择出了N篇文章作为了候选集，并**基于召回的候选集构建了与用户历史相关的特征**，**以及用户本身的属性特征**，**文章本身的属性特征**，以及**用户与文章之间的特征**，下面就是使用机器学习模型来对构造好的特征进行学习，然后对测试集进行预测，得到测试集中的每个候选集用户点击的概率，**返回点击概率最大的topk个文章**，作为最终的结果。

排序阶段选择了三个比较有代表性的排序模型，它们分别是：

1. LGB的排序模型
2. LGB的分类模型
3. 深度学习的分类模型DIN

得到了最终的排序模型输出的结果之后，还选择了两种比较经典的模型集成的方法：

1. **输出结果加权融合**
2. Staking（将模型的输出结果再**使用一个简单模型**进行预测）

In [3]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import gc, os
import time
from datetime import datetime
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

## 读取排序特征

In [4]:
data_path = r'C:\Users\Administrator\RecommendSystem_Data\RecommandNews_data\data_raw\\'
save_path = r'C:\Users\Administrator\RecommendSystem_Data\RecommandNews_data\tmp_results\\'
offline = False

In [5]:
# 重新读取数据的时候，发现click_article_id是一个浮点数，所以将其转换成int类型
# 分别读取，训练，验证，测试数据
trn_user_item_feats_df = pd.read_csv(save_path + 'trn_user_item_feats_df.csv')
trn_user_item_feats_df['click_article_id'] = trn_user_item_feats_df['click_article_id'].astype(int)

if offline:
    val_user_item_feats_df = pd.read_csv(save_path + 'val_user_item_feats_df.csv')
    val_user_item_feats_df['click_article_id'] = val_user_item_feats_df['click_article_id'].astype(int)
else:
    val_user_item_feats_df = None
    
tst_user_item_feats_df = pd.read_csv(save_path + 'tst_user_item_feats_df.csv')
tst_user_item_feats_df['click_article_id'] = tst_user_item_feats_df['click_article_id'].astype(int)

# 做特征的时候为了方便，给测试集也打上了一个无效的标签，这里直接删掉就行
del tst_user_item_feats_df['label']

In [6]:
# datetime.today()

### 返回排序后的结果

In [7]:
def submit(recall_df, topk=5, model_name=None):
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')  #同分不重复排名
    
    # 判断是不是每个用户都有5篇文章及以上
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk
    
    del recall_df['pred_score']
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    """
    注：这里要打开看一下
    """
    
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    # 按照提交格式定义列名
    submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
                                                  3: 'article_3', 4: 'article_4', 5: 'article_5'})
    
    save_name = save_path + model_name + '_' + datetime.today().strftime('%m-%d') + '.csv'
    submit.to_csv(save_name, index=False, header=True)

In [8]:
# 排序结果归一化
def norm_sim(sim_df, weight=0.0):
    # print(sim_df.head())
    min_sim = sim_df.min()
    max_sim = sim_df.max()
    if max_sim == min_sim:
        sim_df = sim_df.apply(lambda sim: 1.0)
    else:
        sim_df = sim_df.apply(lambda sim: 1.0 * (sim - min_sim) / (max_sim - min_sim))

    sim_df = sim_df.apply(lambda sim: sim + weight)  # plus one
    return sim_df

### LGB排序模型

In [9]:
# 防止中间出错之后重新读取数据
trn_user_item_feats_df_rank_model = trn_user_item_feats_df.copy()

if offline:
    val_user_item_feats_df_rank_model = val_user_item_feats_df.copy()
    
tst_user_item_feats_df_rank_model = tst_user_item_feats_df.copy()

In [10]:
# 定义特征列
lgb_cols = ['sim0', 'time_diff0', 'word_diff0','sim_max', 'sim_min', 'sim_sum', 
            'sim_mean', 'score','click_size', 'time_diff_mean', 'active_level',
            'click_environment','click_deviceGroup', 'click_os', 'click_country', 
            'click_region','click_referrer_type', 'user_time_hob1', 'user_time_hob2',
            'words_hbo', 'category_id', 'created_at_ts','words_count']

In [11]:
df = pd.DataFrame({'key1':list('aaabbbaabb'),
				'key2':[1,2,2,1,2,1,1,2,1,2,],
				'data1':np.random.randn(10),
				'data2':np.random.randn(10)})
b1 = df.groupby(['key1','key2'],as_index=False).apply(lambda x:x.iloc[[0,1]])
b1

key1  key2     data1     data2
0 0    a     1 -0.548877 -0.429742
  6    a     1 -0.666732 -1.692905
1 1    a     2  0.685470  0.316658
  2    a     2  0.011107 -0.202242
2 3    b     1  0.690449 -1.233406
  5    b     1 -0.992060 -0.240974
3 4    b     2  1.152560 -0.348993
  9    b     2  0.702055  0.855363

In [12]:
b1.index

MultiIndex([(0, 0),
            (0, 6),
            (1, 1),
            (1, 2),
            (2, 3),
            (2, 5),
            (3, 4),
            (3, 9)],
           )

In [13]:
b2 = df.groupby(['key1','key2'],as_index=True).apply(lambda x:x.iloc[[0,1]])
b2

key1  key2     data1     data2
key1 key2                                 
a    1    0    a     1 -0.548877 -0.429742
          6    a     1 -0.666732 -1.692905
     2    1    a     2  0.685470  0.316658
          2    a     2  0.011107 -0.202242
b    1    3    b     1  0.690449 -1.233406
          5    b     1 -0.992060 -0.240974
     2    4    b     2  1.152560 -0.348993
          9    b     2  0.702055  0.855363

In [14]:
b2.index

MultiIndex([('a', 1, 0),
            ('a', 1, 6),
            ('a', 2, 1),
            ('a', 2, 2),
            ('b', 1, 3),
            ('b', 1, 5),
            ('b', 2, 4),
            ('b', 2, 9)],
           names=['key1', 'key2', None])

In [15]:
b3 = df.groupby(['key1'],as_index=False).apply(lambda x:x.iloc[[0,1]])
b3

key1  key2     data1     data2
0 0    a     1 -0.548877 -0.429742
  1    a     2  0.685470  0.316658
1 3    b     1  0.690449 -1.233406
  4    b     2  1.152560 -0.348993

In [16]:
b3.count()['key1']

4

**注：ax_index=False 可以禁用分组键作为索引的行为，同时自动给定一个索引。**

In [17]:
# 排序模型分组
trn_user_item_feats_df_rank_model.sort_values(by=['user_id'], inplace=True)
g_train = trn_user_item_feats_df_rank_model.groupby(['user_id'], as_index=False).count()["label"].values

if offline:
    val_user_item_feats_df_rank_model.sort_values(by=['user_id'], inplace=True)
    g_val = val_user_item_feats_df_rank_model.groupby(['user_id'], as_index=False).count()["label"].values

In [18]:
# 排序模型定义
"""
注：这里的参数是初始选择没有调参的。
"""
lgb_ranker = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=100, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16)  

In [19]:
lgb_ranker.fit??

In [20]:
# 排序模型训练
"""
eval_metric: Default: 'l2' for LGBMRegressor, 'logloss' for LGBMClassifier, 'ndcg' for LGBMRanker.
eval_at : iterable of int, optional (default=(1, 2, 3, 4, 5))  The evaluation positions of the specified metric.
"""

if offline:
    lgb_ranker.fit(trn_user_item_feats_df_rank_model[lgb_cols], trn_user_item_feats_df_rank_model['label'], group=g_train,
                eval_set=[(val_user_item_feats_df_rank_model[lgb_cols], val_user_item_feats_df_rank_model['label'])], 
                eval_group= [g_val], eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ], early_stopping_rounds=50, )
else:
    lgb_ranker.fit(trn_user_item_feats_df[lgb_cols], trn_user_item_feats_df['label'], group=g_train)

In [21]:
# 模型预测
tst_user_item_feats_df['pred_score'] = lgb_ranker.predict(tst_user_item_feats_df[lgb_cols], num_iteration=lgb_ranker.best_iteration_)
# num_iteration 选择表现最好的参数迭代

# 将这里的排序结果保存一份，用户后面的模型融合
tst_user_item_feats_df[['user_id', 'click_article_id', 'pred_score']].to_csv(save_path + 'lgb_ranker_score.csv', index=False)

In [22]:
# 预测结果重新排序, 及生成提交结果
rank_results = tst_user_item_feats_df[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
submit(rank_results, topk=5, model_name='lgb_ranker')   # 调用 submit 方法，在本地生成提交结果

In [23]:
# 五折交叉验证，这里的五折交叉是以用户为目标进行五折划分
#  这一部分与前面的单独训练和验证是分开的
def get_kfold_users(trn_df, n=5):
    user_ids = trn_df['user_id'].unique()
    user_set = [user_ids[i::n] for i in range(n)]   # 这个取 5 折的方法很好
    return user_set

k_fold = 5
trn_df = trn_user_item_feats_df_rank_model
user_set = get_kfold_users(trn_df, n=k_fold)

score_list = []
score_df = trn_df[['user_id', 'click_article_id','label']]
sub_preds = np.zeros(tst_user_item_feats_df_rank_model.shape[0])

# 五折交叉验证，并将中间结果保存用于staking
for n_fold, valid_user in enumerate(user_set):
    train_idx = trn_df[~trn_df['user_id'].isin(valid_user)] # add slide user   # 选出不在 5 折中的其他用户，作为训练用户
    valid_idx = trn_df[trn_df['user_id'].isin(valid_user)]
    
    # 训练集与验证集的用户分组
    train_idx.sort_values(by=['user_id'], inplace=True)
    g_train = train_idx.groupby(['user_id'], as_index=False).count()["label"].values
    print("g_train :" , g_train)  # g_train : [1 1 1 ... 1 1 3]  是这样的形式

    
    valid_idx.sort_values(by=['user_id'], inplace=True)
    g_val = valid_idx.groupby(['user_id'], as_index=False).count()["label"].values
    print("g_val :" , g_val)  # g_val : [1 1 1 ... 1 1 1]  是这样的形式
    
    # 定义模型
    lgb_ranker = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=100, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16)  
    # 训练模型
    lgb_ranker.fit(train_idx[lgb_cols], train_idx['label'], group=g_train,
                   eval_set=[(valid_idx[lgb_cols], valid_idx['label'])], eval_group= [g_val], 
                   eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ], early_stopping_rounds=50, )
    
    # 预测验证集结果
    valid_idx['pred_score'] = lgb_ranker.predict(valid_idx[lgb_cols], num_iteration=lgb_ranker.best_iteration_)
    
    # 对输出结果进行归一化
    valid_idx['pred_score'] = valid_idx[['pred_score']].transform(lambda x: norm_sim(x))
    
    valid_idx.sort_values(by=['user_id', 'pred_score'])
    valid_idx['pred_rank'] = valid_idx.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 将验证集的预测结果放到一个列表中，后面进行拼接
    score_list.append(valid_idx[['user_id', 'click_article_id', 'pred_score', 'pred_rank']])
    
    # 如果是线上测试，需要计算每次交叉验证的结果相加，最后求平均
    if not offline:
        print("++++++++++++++++++++++++")
        sub_preds += lgb_ranker.predict(tst_user_item_feats_df_rank_model[lgb_cols], lgb_ranker.best_iteration_)
        
score_df_ = pd.concat(score_list, axis=0)
score_df = score_df.merge(score_df_, how='left', on=['user_id', 'click_article_id'])  # 这里是为什么？
# 保存训练集交叉验证产生的新特征
score_df[['user_id', 'click_article_id', 'pred_score', 'pred_rank', 'label']].to_csv(save_path + 'trn_lgb_ranker_feats.csv', index=False)
    
# 测试集的预测结果，多次交叉验证求平均,将预测的score和对应的rank特征保存，可以用于后面的staking，这里还可以构造其他更多的特征
tst_user_item_feats_df_rank_model['pred_score'] = sub_preds / k_fold
tst_user_item_feats_df_rank_model['pred_score'] = tst_user_item_feats_df_rank_model['pred_score'].transform(lambda x: norm_sim(x))  # norm_sim 把0全都转为1了
tst_user_item_feats_df_rank_model.sort_values(by=['user_id', 'pred_score'])
tst_user_item_feats_df_rank_model['pred_rank'] = tst_user_item_feats_df_rank_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

# 保存测试集交叉验证的新特征
tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'tst_lgb_ranker_feats.csv', index=False)

g_train : [1 1 1 ... 1 3 1]
g_val : [1 1 1 ... 1 1 1]
[1]	valid_0's ndcg@1: 0.99975	valid_0's ndcg@2: 0.999908	valid_0's ndcg@3: 0.999908	valid_0's ndcg@4: 0.999908	valid_0's ndcg@5: 0.999908
Training until validation scores don't improve for 50 rounds
[2]	valid_0's ndcg@1: 0.99975	valid_0's ndcg@2: 0.999908	valid_0's ndcg@3: 0.999908	valid_0's ndcg@4: 0.999908	valid_0's ndcg@5: 0.999908
[3]	valid_0's ndcg@1: 0.99975	valid_0's ndcg@2: 0.999908	valid_0's ndcg@3: 0.999908	valid_0's ndcg@4: 0.999908	valid_0's ndcg@5: 0.999908
[4]	valid_0's ndcg@1: 0.99975	valid_0's ndcg@2: 0.999908	valid_0's ndcg@3: 0.999908	valid_0's ndcg@4: 0.999908	valid_0's ndcg@5: 0.999908
[5]	valid_0's ndcg@1: 0.99975	valid_0's ndcg@2: 0.999908	valid_0's ndcg@3: 0.999908	valid_0's ndcg@4: 0.999908	valid_0's ndcg@5: 0.999908
[6]	valid_0's ndcg@1: 0.99975	valid_0's ndcg@2: 0.999908	valid_0's ndcg@3: 0.999908	valid_0's ndcg@4: 0.999908	valid_0's ndcg@5: 0.999908
[7]	valid_0's ndcg@1: 0.99975	valid_0's ndcg@2: 0.999908	

[8]	valid_0's ndcg@1: 0.999775	valid_0's ndcg@2: 0.999917	valid_0's ndcg@3: 0.999917	valid_0's ndcg@4: 0.999917	valid_0's ndcg@5: 0.999917
[9]	valid_0's ndcg@1: 0.999775	valid_0's ndcg@2: 0.999917	valid_0's ndcg@3: 0.999917	valid_0's ndcg@4: 0.999917	valid_0's ndcg@5: 0.999917
[10]	valid_0's ndcg@1: 0.999775	valid_0's ndcg@2: 0.999917	valid_0's ndcg@3: 0.999917	valid_0's ndcg@4: 0.999917	valid_0's ndcg@5: 0.999917
[11]	valid_0's ndcg@1: 0.999775	valid_0's ndcg@2: 0.999917	valid_0's ndcg@3: 0.999917	valid_0's ndcg@4: 0.999917	valid_0's ndcg@5: 0.999917
[12]	valid_0's ndcg@1: 0.999775	valid_0's ndcg@2: 0.999917	valid_0's ndcg@3: 0.999917	valid_0's ndcg@4: 0.999917	valid_0's ndcg@5: 0.999917
[13]	valid_0's ndcg@1: 0.999775	valid_0's ndcg@2: 0.999917	valid_0's ndcg@3: 0.999917	valid_0's ndcg@4: 0.999917	valid_0's ndcg@5: 0.999917
[14]	valid_0's ndcg@1: 0.999775	valid_0's ndcg@2: 0.999917	valid_0's ndcg@3: 0.999917	valid_0's ndcg@4: 0.999917	valid_0's ndcg@5: 0.999917
[15]	valid_0's ndcg@1:

[24]	valid_0's ndcg@1: 0.999725	valid_0's ndcg@2: 0.999899	valid_0's ndcg@3: 0.999899	valid_0's ndcg@4: 0.999899	valid_0's ndcg@5: 0.999899
[25]	valid_0's ndcg@1: 0.999725	valid_0's ndcg@2: 0.999899	valid_0's ndcg@3: 0.999899	valid_0's ndcg@4: 0.999899	valid_0's ndcg@5: 0.999899
[26]	valid_0's ndcg@1: 0.999725	valid_0's ndcg@2: 0.999899	valid_0's ndcg@3: 0.999899	valid_0's ndcg@4: 0.999899	valid_0's ndcg@5: 0.999899
[27]	valid_0's ndcg@1: 0.999725	valid_0's ndcg@2: 0.999899	valid_0's ndcg@3: 0.999899	valid_0's ndcg@4: 0.999899	valid_0's ndcg@5: 0.999899
[28]	valid_0's ndcg@1: 0.999725	valid_0's ndcg@2: 0.999899	valid_0's ndcg@3: 0.999899	valid_0's ndcg@4: 0.999899	valid_0's ndcg@5: 0.999899
[29]	valid_0's ndcg@1: 0.999725	valid_0's ndcg@2: 0.999899	valid_0's ndcg@3: 0.999899	valid_0's ndcg@4: 0.999899	valid_0's ndcg@5: 0.999899
[30]	valid_0's ndcg@1: 0.999725	valid_0's ndcg@2: 0.999899	valid_0's ndcg@3: 0.999899	valid_0's ndcg@4: 0.999899	valid_0's ndcg@5: 0.999899
[31]	valid_0's ndcg@

[35]	valid_0's ndcg@1: 0.999875	valid_0's ndcg@2: 0.999954	valid_0's ndcg@3: 0.999954	valid_0's ndcg@4: 0.999954	valid_0's ndcg@5: 0.999954
[36]	valid_0's ndcg@1: 0.999875	valid_0's ndcg@2: 0.999954	valid_0's ndcg@3: 0.999954	valid_0's ndcg@4: 0.999954	valid_0's ndcg@5: 0.999954
[37]	valid_0's ndcg@1: 0.999875	valid_0's ndcg@2: 0.999954	valid_0's ndcg@3: 0.999954	valid_0's ndcg@4: 0.999954	valid_0's ndcg@5: 0.999954
[38]	valid_0's ndcg@1: 0.999875	valid_0's ndcg@2: 0.999954	valid_0's ndcg@3: 0.999954	valid_0's ndcg@4: 0.999954	valid_0's ndcg@5: 0.999954
[39]	valid_0's ndcg@1: 0.999875	valid_0's ndcg@2: 0.999954	valid_0's ndcg@3: 0.999954	valid_0's ndcg@4: 0.999954	valid_0's ndcg@5: 0.999954
[40]	valid_0's ndcg@1: 0.999875	valid_0's ndcg@2: 0.999954	valid_0's ndcg@3: 0.999954	valid_0's ndcg@4: 0.999954	valid_0's ndcg@5: 0.999954
[41]	valid_0's ndcg@1: 0.999875	valid_0's ndcg@2: 0.999954	valid_0's ndcg@3: 0.999954	valid_0's ndcg@4: 0.999954	valid_0's ndcg@5: 0.999954
[42]	valid_0's ndcg@

[43]	valid_0's ndcg@1: 0.99965	valid_0's ndcg@2: 0.999871	valid_0's ndcg@3: 0.999871	valid_0's ndcg@4: 0.999871	valid_0's ndcg@5: 0.999871
[44]	valid_0's ndcg@1: 0.99965	valid_0's ndcg@2: 0.999871	valid_0's ndcg@3: 0.999871	valid_0's ndcg@4: 0.999871	valid_0's ndcg@5: 0.999871
[45]	valid_0's ndcg@1: 0.99965	valid_0's ndcg@2: 0.999871	valid_0's ndcg@3: 0.999871	valid_0's ndcg@4: 0.999871	valid_0's ndcg@5: 0.999871
[46]	valid_0's ndcg@1: 0.99965	valid_0's ndcg@2: 0.999871	valid_0's ndcg@3: 0.999871	valid_0's ndcg@4: 0.999871	valid_0's ndcg@5: 0.999871
[47]	valid_0's ndcg@1: 0.99965	valid_0's ndcg@2: 0.999871	valid_0's ndcg@3: 0.999871	valid_0's ndcg@4: 0.999871	valid_0's ndcg@5: 0.999871
[48]	valid_0's ndcg@1: 0.99965	valid_0's ndcg@2: 0.999871	valid_0's ndcg@3: 0.999871	valid_0's ndcg@4: 0.999871	valid_0's ndcg@5: 0.999871
[49]	valid_0's ndcg@1: 0.99965	valid_0's ndcg@2: 0.999871	valid_0's ndcg@3: 0.999871	valid_0's ndcg@4: 0.999871	valid_0's ndcg@5: 0.999871
[50]	valid_0's ndcg@1: 0.99

In [24]:
# 预测结果重新排序, 及生成提交结果
# 单模型生成提交结果
rank_results = tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
submit(rank_results, topk=5, model_name='lgb_ranker')

### LGB分类模型

In [25]:
# 模型及参数的定义
lgb_Classfication = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=500, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16, verbose=10)  

In [26]:
# 模型训练
if offline:
    lgb_Classfication.fit(trn_user_item_feats_df_rank_model[lgb_cols], trn_user_item_feats_df_rank_model['label'],
                    eval_set=[(val_user_item_feats_df_rank_model[lgb_cols], val_user_item_feats_df_rank_model['label'])], 
                    eval_metric=['auc', ],early_stopping_rounds=50, )
else:
    lgb_Classfication.fit(trn_user_item_feats_df_rank_model[lgb_cols], trn_user_item_feats_df_rank_model['label'])

[LightGBM] [Info] Number of positive: 89, number of negative: 206488
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.042934
[LightGBM] [Debug] init for col-wise cost 0.000016 seconds, init for row-wise cost 0.027695 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038452 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4113
[LightGBM] [Info] Number of data points in the train set: 206577, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000431 -> initscore=-7.749361
[LightGBM] [Info] Start training from score -7.749361
[LightGBM] [Debug] Re-bagging, using 144808 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-

[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144620 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144311 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144677 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144282 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144522 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144979 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144612 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144440 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144699 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[LightGBM] [Debug] Re-bagging, using 144697 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144804 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144692 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144648 data to train
[LightGBM] [Warning] No fur

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144856 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144311 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144696 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144229 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144376 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144699 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144849 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144786 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144295 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144578 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144694 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144818 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144671 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144680 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 144356 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

In [27]:
# 模型预测
tst_user_item_feats_df['pred_score'] = lgb_Classfication.predict_proba(tst_user_item_feats_df[lgb_cols])[:,1]

# 将这里的排序结果保存一份，用户后面的模型融合
tst_user_item_feats_df[['user_id', 'click_article_id', 'pred_score']].to_csv(save_path + 'lgb_cls_score.csv', index=False)

In [28]:
# 预测结果重新排序, 及生成提交结果
rank_results = tst_user_item_feats_df[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
submit(rank_results, topk=5, model_name='lgb_cls')

In [29]:
# 五折交叉验证，这里的五折交叉是以用户为目标进行五折划分
#  这一部分与前面的单独训练和验证是分开的
def get_kfold_users(trn_df, n=5):
    user_ids = trn_df['user_id'].unique()
    user_set = [user_ids[i::n] for i in range(n)]
    return user_set

k_fold = 5
trn_df = trn_user_item_feats_df_rank_model
user_set = get_kfold_users(trn_df, n=k_fold)

score_list = []
score_df = trn_df[['user_id', 'click_article_id', 'label']]
sub_preds = np.zeros(tst_user_item_feats_df_rank_model.shape[0])

# 五折交叉验证，并将中间结果保存用于staking
for n_fold, valid_user in enumerate(user_set):
    train_idx = trn_df[~trn_df['user_id'].isin(valid_user)] # add slide user
    valid_idx = trn_df[trn_df['user_id'].isin(valid_user)]
    
    # 模型及参数的定义
    lgb_Classfication = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=100, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16, verbose=10)  
    # 训练模型
    lgb_Classfication.fit(train_idx[lgb_cols], train_idx['label'],eval_set=[(valid_idx[lgb_cols], valid_idx['label'])], 
                          eval_metric=['auc', ],early_stopping_rounds=50, )
    
    # 预测验证集结果
    valid_idx['pred_score'] = lgb_Classfication.predict_proba(valid_idx[lgb_cols], 
                                                              num_iteration=lgb_Classfication.best_iteration_)[:,1]
    
    # 对输出结果进行归一化 分类模型输出的值本身就是一个概率值不需要进行归一化
    # valid_idx['pred_score'] = valid_idx[['pred_score']].transform(lambda x: norm_sim(x))
    
    valid_idx.sort_values(by=['user_id', 'pred_score'])
    valid_idx['pred_rank'] = valid_idx.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 将验证集的预测结果放到一个列表中，后面进行拼接
    score_list.append(valid_idx[['user_id', 'click_article_id', 'pred_score', 'pred_rank']])
    
    # 如果是线上测试，需要计算每次交叉验证的结果相加，最后求平均
    if not offline:
        sub_preds += lgb_Classfication.predict_proba(tst_user_item_feats_df_rank_model[lgb_cols], 
                                                     num_iteration=lgb_Classfication.best_iteration_)[:,1]
    
score_df_ = pd.concat(score_list, axis=0)
score_df = score_df.merge(score_df_, how='left', on=['user_id', 'click_article_id'])
# 保存训练集交叉验证产生的新特征
score_df[['user_id', 'click_article_id', 'pred_score', 'pred_rank', 'label']].to_csv(save_path + 'trn_lgb_cls_feats.csv', index=False)
    
# 测试集的预测结果，多次交叉验证求平均,将预测的score和对应的rank特征保存，可以用于后面的staking，这里还可以构造其他更多的特征
tst_user_item_feats_df_rank_model['pred_score'] = sub_preds / k_fold
tst_user_item_feats_df_rank_model['pred_score'] = tst_user_item_feats_df_rank_model['pred_score'].transform(lambda x: norm_sim(x))
tst_user_item_feats_df_rank_model.sort_values(by=['user_id', 'pred_score'])
tst_user_item_feats_df_rank_model['pred_rank'] = tst_user_item_feats_df_rank_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

# 保存测试集交叉验证的新特征
tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'tst_lgb_cls_feats.csv', index=False)

[LightGBM] [Info] Number of positive: 71, number of negative: 165222
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.042924
[LightGBM] [Debug] init for col-wise cost 0.000015 seconds, init for row-wise cost 0.019101 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027359 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4106
[LightGBM] [Info] Number of data points in the train set: 165293, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000430 -> initscore=-7.752365
[LightGBM] [Info] Start training from score -7.752365
[LightGBM] [Debug] Re-bagging, using 115698 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[1]	valid_0's auc: 0.5

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[23]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00676031
[LightGBM] [Debug] Re-bagging, using 116226 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[24]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00676031
[LightGBM] [Debug] Re-bagging, using 115800 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[25]	valid_0's auc: 0

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[49]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00676031
[LightGBM] [Debug] Re-bagging, using 115641 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[50]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00676031
[LightGBM] [Debug] Re-bagging, using 115542 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[51]	valid_0's auc: 0

[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[31]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00754063
[LightGBM] [Debug] Re-bagging, using 115816 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[32]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00754063
[LightGBM] [Debug] Re-bagging, using 115670 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[33]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00754063
[LightGBM] [Debug] Re-bagging, usin

[LightGBM] [Debug] Re-bagging, using 115660 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[5]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00560552
[LightGBM] [Debug] Re-bagging, using 115904 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[6]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00560552
[LightGBM] [Debug] Re-bagging, using 115639 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leave

[LightGBM] [Debug] Re-bagging, using 115782 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[30]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00560552
[LightGBM] [Debug] Re-bagging, using 115535 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[31]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00560552
[LightGBM] [Debug] Re-bagging, using 115807 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more lea

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[3]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00407802
[LightGBM] [Debug] Re-bagging, using 115525 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[4]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00407802
[LightGBM] [Debug] Re-bagging, using 115633 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[5]	valid_0's auc: 0.5	

[LightGBM] [Debug] Re-bagging, using 115779 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[32]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00407802
[LightGBM] [Debug] Re-bagging, using 115660 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[33]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00407802
[LightGBM] [Debug] Re-bagging, using 115305 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more lea

[LightGBM] [Debug] Re-bagging, using 115207 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[3]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00948015
[LightGBM] [Debug] Re-bagging, using 115448 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[4]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00948015
[LightGBM] [Debug] Re-bagging, using 115553 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leave

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[37]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00948015
[LightGBM] [Debug] Re-bagging, using 115392 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[38]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.00948015
[LightGBM] [Debug] Re-bagging, using 115754 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and max_depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[39]	valid_0's auc: 0

In [30]:
# 预测结果重新排序, 及生成提交结果
rank_results = tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
submit(rank_results, topk=5, model_name='lgb_cls')

## DIN模型

### 用户的历史点击行为列表

这个是为后面的DIN模型服务的

In [31]:
if offline:
    all_data = pd.read_csv(data_path + 'train_click_log.csv')
else:
    trn_data = pd.read_csv(data_path + 'train_click_log.csv')
    tst_data = pd.read_csv(data_path + 'testA_click_log.csv')
    all_data = trn_data.append(tst_data)

In [32]:
hist_click =all_data[['user_id', 'click_article_id']].groupby('user_id').agg({list}).reset_index()
his_behavior_df = pd.DataFrame()
his_behavior_df['user_id'] = hist_click['user_id']
his_behavior_df['hist_click_article_id'] = hist_click['click_article_id']

In [33]:
trn_user_item_feats_df.head(1)

,user_id,click_article_id,sim0,time_diff0,word_diff0,sim_max,sim_min,sim_sum,sim_mean,score,...,click_country,click_region,click_referrer_type,user_time_hob1,user_time_hob2,words_hbo,category_id,created_at_ts,words_count,is_cat_hab
0,0,16800,0.238681,260897000,48,0.238681,0.238681,0.238681,0.238681,-108.0,...,1,25,2,0.343715,0.992865,266.0,7,1508445988000,210,0


In [34]:
trn_user_item_feats_df_din_model = trn_user_item_feats_df.copy()

if offline:
    val_user_item_feats_df_din_model = val_user_item_feats_df.copy()
else: 
    val_user_item_feats_df_din_model = None
    
tst_user_item_feats_df_din_model = tst_user_item_feats_df.copy()

In [35]:
trn_user_item_feats_df_din_model = trn_user_item_feats_df_din_model.merge(his_behavior_df, on='user_id')

if offline:
    val_user_item_feats_df_din_model = val_user_item_feats_df_din_model.merge(his_behavior_df, on='user_id')
else:
    val_user_item_feats_df_din_model = None

tst_user_item_feats_df_din_model = tst_user_item_feats_df_din_model.merge(his_behavior_df, on='user_id')

### DIN模型简介

我们下面尝试使用DIN模型， DIN的全称是Deep Interest Network， 这是阿里2018年基于前面的深度学习模型无法表达用户多样化的兴趣而提出的一个模型， **它可以通过考虑【给定的候选广告】和【用户的历史行为】的相关性，来计算用户兴趣的表示向量**。具体来说就是**通过引入局部激活单元，通过软搜索历史行为的相关部分来关注相关的用户兴趣，并采用加权和来获得有关候选广告的用户兴趣的表示**。**与候选广告相关性较高的行为会获得较高的激活权重，并支配着用户兴趣**。该表示向量在不同广告上有所不同，大大提高了模型的表达能力。所以该模型对于此次新闻推荐的任务也比较适合， 我们在这里通过当前的候选文章与用户历史点击文章的相关性来计算用户对于文章的兴趣。 该模型的结构如下：

![image-20201116201646983](http://ryluo.oss-cn-chengdu.aliyuncs.com/abc/image-20201116201646983.png)


我们这里直接调包来使用这个模型， 关于这个模型的详细细节部分我们会在下一期的推荐系统组队学习中给出。下面说一下该模型如何具体使用：deepctr的函数原型如下：

> def DIN(dnn_feature_columns, history_feature_list, dnn_use_bn=False,
>     dnn_hidden_units=(200, 80), dnn_activation='relu', att_hidden_size=(80, 40), att_activation="dice",
>    att_weight_normalization=False, l2_reg_dnn=0, l2_reg_embedding=1e-6, dnn_dropout=0, seed=1024,
>     task='binary'):
>
> * dnn_feature_columns: 特征列， 包含数据所有特征的列表
> * history_feature_list: 用户历史行为列， 反应用户历史行为的特征的列表
> * dnn_use_bn: 是否使用BatchNormalization
> * dnn_hidden_units: 全连接层网络的层数和每一层神经元的个数， 一个列表或者元组
> * dnn_activation_relu: 全连接网络的激活单元类型
> * att_hidden_size: 注意力层的全连接网络的层数和每一层神经元的个数
> * att_activation: 注意力层的激活单元类型
> * att_weight_normalization: 是否归一化注意力得分
> * l2_reg_dnn: 全连接网络的正则化系数
> * l2_reg_embedding: embedding向量的正则化稀疏
> * dnn_dropout: 全连接网络的神经元的失活概率
> * task: 任务， 可以是分类， 也可是是回归

在具体使用的时候， 我们必须要传入特征列和历史行为列， 但是再传入之前， 我们需要进行一下特征列的预处理。具体如下：

1. 首先，我们要处理数据集， 得到数据， 由于我们是基于用户过去的行为去预测用户是否点击当前文章， 所以**我们需要把数据的特征列划分成数值型特征， 离散型特征和历史行为特征列三部分**， 对于每一部分， DIN模型的处理会有不同
   1. 对于离散型特征， 在我们的数据集中就是那些类别型的特征， 比如user_id这种， 这种类别型特征， 我们**首先要经过embedding处理得到每个特征的低维稠密型表示**， 既然要经过embedding， 那么我们就需要为每一列的类别特征的取值建立一个字典，并指明embedding维度， 所以在使用deepctr的DIN模型准备数据的时候， **我们需要通过SparseFeat函数指明这些类别型特征, 这个函数的传入参数就是列名， 列的唯一取值(建立字典用)和embedding维度**。
   2. 对于用户历史行为特征列， 比如文章id， 文章的类别等这种， 同样的我们需要先经过embedding处理， 只不过和上面不一样的地方是，对于这种特征， 我们在得到每个特征的embedding表示之后， 还需要通过一个Attention_layer**计算用户的历史行为**和**当前候选文章的相关性**以此得到当前用户的embedding向量， 这个向量就可以**基于当前的候选文章**与**用户过去点击过得历史文章的相似性**的程度来反应用户的兴趣， 并且随着用户的不同的历史点击来变化，去动态的模拟用户兴趣的变化过程。这类特征对于每个用户都是一个历史行为序列， 对于每个用户， 历史行为序列长度会不一样， 可能有的用户点击的历史文章多，有的点击的历史文章少， **所以我们还需要把这个长度统一起来**， 在为DIN模型准备数据的时候， 我们首先要通过SparseFeat函数指明这些类别型特征， 然后还需要通过VarLenSparseFeat函数再进行序列填充， 使得每个用户的历史序列一样长， 所以这个函数参数中会有个maxlen，来指明序列的最大长度是多少。
   3. **对于连续型特征列， 我们只需要用DenseFeat函数来指明列名和维度即可**。
2. 处理完特征列之后， 我们把相应的数据与列进行对应，就得到了最后的数据。

下面根据具体的代码感受一下， 逻辑是这样， 首先我们需要写一个数据准备函数， 在这里面就是根据上面的具体步骤准备数据， 得到数据和特征列， 然后就是建立DIN模型并训练， 最后基于模型进行测试。

In [36]:
# 导入deepctr
from deepctr.models import DIN
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [37]:
# 数据准备函数
def get_din_feats_columns(df, dense_fea, sparse_fea, behavior_fea, his_behavior_fea, emb_dim=32, max_len=100):
    """
    数据准备函数:
    df: 数据集
    dense_fea: 数值型特征列
    sparse_fea: 离散型特征列
    behavior_fea: 用户的候选行为特征列
    his_behavior_fea: 用户的历史行为特征列
    embedding_dim: embedding的维度， 这里为了简单， 统一把离散型特征列采用一样的隐向量维度
    max_len: 用户序列的最大长度
    """
    
    sparse_feature_columns = [SparseFeat(feat, vocabulary_size=df[feat].nunique() + 1, embedding_dim=emb_dim) for feat in sparse_fea]
    "我终于理解这个是干什么用的了。。"
    print(sparse_feature_columns)
    
    dense_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_fea]
    print(dense_feature_columns)
    
    var_feature_columns = [VarLenSparseFeat(SparseFeat(feat, vocabulary_size=df['click_article_id'].nunique() + 1,
                                    embedding_dim=emb_dim, embedding_name='click_article_id'), maxlen=max_len) for feat in hist_behavior_fea]
    
    dnn_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns
    
    # 建立x, x是一个字典的形式
    x = {}
    for name in get_feature_names(dnn_feature_columns):
        if name in his_behavior_fea:
            # 这是历史行为序列
            his_list = [l for l in df[name]]
            x[name] = pad_sequences(his_list, maxlen=max_len, padding='post')      # 二维数组   # 这里是把他拉长么？
        else:
            x[name] = df[name].values
    
    return x, dnn_feature_columns

In [38]:
# 把特征分开
"""
这里是根据项目现实情况，主动选择的
"""

sparse_fea = ['user_id', 'click_article_id', 'category_id', 'click_environment', 'click_deviceGroup', 
              'click_os', 'click_country', 'click_region', 'click_referrer_type', 'is_cat_hab']

behavior_fea = ['click_article_id']

hist_behavior_fea = ['hist_click_article_id']

dense_fea = ['sim0', 'time_diff0', 'word_diff0', 'sim_max', 'sim_min', 'sim_sum', 'sim_mean', 'score',
             'rank','click_size','time_diff_mean','active_level','user_time_hob1','user_time_hob2',
             'words_hbo','words_count']

In [39]:
# dense特征进行归一化, 神经网络训练都需要将数值进行归一化处理
mm = MinMaxScaler()

# 下面是做一些特殊处理，当在其他的地方出现无效值的时候，不处理无法进行归一化，刚开始可以先把他注释掉，在运行了下面的代码
# 之后如果发现报错，应该先去想办法处理如何不出现inf之类的值
# trn_user_item_feats_df_din_model.replace([np.inf, -np.inf], 0, inplace=True)
# tst_user_item_feats_df_din_model.replace([np.inf, -np.inf], 0, inplace=True)

"""
转换 训练，验证，测试 3部分数据
"""
for feat in dense_fea:
    trn_user_item_feats_df_din_model[feat] = mm.fit_transform(trn_user_item_feats_df_din_model[[feat]])
    
    if val_user_item_feats_df_din_model is not None:
        val_user_item_feats_df_din_model[feat] = mm.fit_transform(val_user_item_feats_df_din_model[[feat]])
    
    tst_user_item_feats_df_din_model[feat] = mm.fit_transform(tst_user_item_feats_df_din_model[[feat]])

In [40]:
# 准备训练数据
x_trn, dnn_feature_columns = get_din_feats_columns(trn_user_item_feats_df_din_model, dense_fea, 
                                               sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)
y_trn = trn_user_item_feats_df_din_model['label'].values

if offline:
    # 准备验证数据
    x_val, dnn_feature_columns = get_din_feats_columns(val_user_item_feats_df_din_model, dense_fea, 
                                                   sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)
    y_val = val_user_item_feats_df_din_model['label'].values
    
dense_fea = [x for x in dense_fea if x != 'label']
x_tst, dnn_feature_columns = get_din_feats_columns(tst_user_item_feats_df_din_model, dense_fea, 
                                               sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


[SparseFeat(name='user_id', vocabulary_size=200001, embedding_dim=32, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x0000027BAEF7B070>, embedding_name='user_id', group_name='default_group', trainable=True), SparseFeat(name='click_article_id', vocabulary_size=7030, embedding_dim=32, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x0000027BAEF65220>, embedding_name='click_article_id', group_name='default_group', trainable=True), SparseFeat(name='category_id', vocabulary_size=195, embedding_dim=32, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x0000027BAEF65DC0>, embedding_name='category_id', group_name='default_group', trainable=True), SparseFeat(name='click_environment', vocabulary_size=4, embedding_dim=32, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.ker

In [41]:
# 建立模型
model = DIN(dnn_feature_columns, behavior_fea)

# 查看模型结构
model.summary()

# 模型编译
model.compile('adam', 'binary_crossentropy',metrics=['binary_crossentropy', tf.keras.metrics.AUC()])

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'attention_sequence_pooling_layer/local_activation_unit/kernel:0' shape=(40, 1) dtype=float32>
  <tf.Variable 'attention_sequence_pooling_layer/local_activation_unit/bias:0' shape=(1,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'attention_sequence_pooling_layer/local_activation_unit/kernel:0' shape=(40, 1) dtype=float32>
  <tf.Variable 'attention_sequence_pooling_layer/local_activation_unit/bias:0' shape=(1,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
click_article_id (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
category_id (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
click_environment (InputLayer)  [(None, 1)]          0                                            
______________________________________________________________________________________________

In [42]:
# 模型训练
if offline:
    history = model.fit(x_trn, y_trn, verbose=1, epochs=10, validation_data=(x_val, y_val) , batch_size=256)
else:
    # 也可以使用上面的语句用自己采样出来的验证集
    # history = model.fit(x_trn, y_trn, verbose=1, epochs=3, validation_split=0.3, batch_size=256)
    print("here...")
    history = model.fit(x_trn, y_trn, verbose=1, epochs=2, batch_size=256)

here...
Epoch 1/2


InvalidArgumentError:  indices[7,0] = 247 is not in [0, 174)
	 [[node model/sparse_emb_category_id/embedding_lookup (defined at <ipython-input-42-c7915fffe6a3>:8) ]] [Op:__inference_train_function_5140]

Errors may have originated from an input operation.
Input Source operations connected to node model/sparse_emb_category_id/embedding_lookup:
 model/sparse_emb_category_id/embedding_lookup/3436 (defined at C:\ProgramData\Anaconda3\lib\contextlib.py:113)

Function call stack:
train_function
